In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.nn import convolution

from celluloid import Camera
from matplotlib import animation, rc
from IPython.display import HTML
from IPython.display import clear_output
from time import time
from scipy.special import factorial

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from FiniteDifferences.Maxwell2d import Maxwell2DFiniteDifference

## A free electromagnetic wave in a box.

In [4]:
mwell = Maxwell2DFiniteDifference(mesh_size=0.01, step_size=1e-3, frame_rate=60)

In [5]:
charge_density_callable = lambda x,y: 0
electric_field_callable = lambda x,y: np.zeros((2,))
magnetic_field_callable = lambda x, y: np.exp(-np.sum(np.square((np.array([x - 0.5, y- 0.5]).reshape((2,1,1)))/0.1), axis=0))

In [ ]:
B, E, I, p = mwell.evolve(magnetic_field_callable, 
                          electric_field_callable, 
                          charge_density_callable, 
                          integration_period=1,
                          order=20,
                          video=True)

## A coin added to the box.

In [ ]:
charge_density_callable = lambda x,y: 0
electric_field_callable = lambda x,y: np.zeros((2,))
magnetic_field_callable = lambda x, y: np.exp(-np.sum(np.square((np.array([x - 0.5, y- 0.5]).reshape((2,1,1)))/0.1), axis=0))
resistance = lambda x, y: 0.01 if np.sqrt((x - 0.2)**2 + (y-0.2)**2) < 0.1 else np.inf

In [ ]:
mwell = Maxwell2DFiniteDifference(mesh_size=0.001, 
                                  step_size=1e-3, 
                                  resistance=resistance, 
                                  frame_rate=20)

In [ ]:
B, E, I, p = mwell.evolve(magnetic_field_callable, 
                          electric_field_callable, 
                          charge_density_callable, 
                          integration_period=1,
                          order=5,
                          video=True)

## Dipole antenna

In [ ]:
charge_density_callable = lambda x,y: 0
electric_field_callable = lambda x,y: np.zeros((2,))
magnetic_field_callable = lambda x, y: np.exp(-np.sum(np.square((np.array([x - 0.3, y- 0.5]).reshape((2,1,1)))/0.1), axis=0)) - \
                                       np.exp(-np.sum(np.square((np.array([x - 0.7, y- 0.5]).reshape((2,1,1)))/0.1), axis=0))

In [ ]:
def antenna(x, y):
    if 0.45 < x < 0.55 and 0.2 < y < 0.8:
        return 0.01
    else:
        return np.inf

In [ ]:
mwell = Maxwell2DFiniteDifference(mesh_size=0.001, 
                                  step_size=1e-3, 
                                  resistance=antenna, 
                                  frame_rate=20)

In [ ]:
B, E, I, p = mwell.evolve(magnetic_field_callable, 
                          electric_field_callable, 
                          charge_density_callable, 
                          integration_period=1,
                          order=5,
                          video=True)

In [ ]:
plt.imshow(mwell._R)

In [ ]:
plt.imshow(p, vmin=-5e1,vmax=5e1)

In [ ]:
np.max(p)